<a href="https://colab.research.google.com/github/usma11dia0/web_scraping_on_colab/blob/main/web_scraping_on_colab_minkabu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# GoogleColab上にあるデフォルトパッケージの依存関係を整理
import pkg_resources
import importlib

importlib.reload(pkg_resources)

<module 'pkg_resources' from '/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py'>

In [ ]:
# 環境構築 (スクレイピング)
%pip install -q google-colab-selenium

# 環境構築 (形態素解析)
import numpy
!pip install ginza==5.1.3 ja_ginza_electra==5.1.3 https://github.com/megagonlabs/ginza/releases/download/v5.1.3/ja_ginza_electra-5.1.3-with-model.tar.gz
!pip install "spacy>=3.2.0,<3.7.0"
!pip install "spacy[cuda117]"
!pip install fastapi==0.109.2 kaleido==0.2.1 python-multipart==0.0.9 uvicorn==0.27.1
!pip install japanize-matplotlib==1.1.3
!pip install pyvis==0.3.2


     ━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/9.9 MB 14.8 MB/s eta 0:00:01

In [4]:
# python検索パスを追加
import sys

# 追加したいパス
paths_to_add = [
    '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/financial_service_agency/'
]

for path in paths_to_add:
    if path not in sys.path:
        sys.path.append(path)

In [5]:
#環境依存値設定
import os
os.environ['ENV'] = 'local'
os.environ['TARGET'] = 'みんかぶ'
os.environ['COMPANY'] = 'さくらインターネット'

print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/financial_service_agency/']


In [6]:
#標準ライブラリ
import os
import math
import json
import re
import time
from datetime import datetime

#内部ライブラリ
from path_config import WORKING_DIR
from env.get_env import APP_ENV
from config.get_config import APP_CONFIG
from utils.scraping_functions import scroll_page
from utils.nlp_utils import extract_trend_comment, classify_trend_word

#サードパーティライブラリ
import spacy
import pandas as pd
import google_colab_selenium as gs
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from logging import (
    getLogger,
    StreamHandler,
    DEBUG,
    INFO,
    Formatter,
    config
)

In [7]:
#環境値読み込み
TARGET_URL = APP_CONFIG['URL'][os.environ['TARGET']]

# 一時対応
STOCK_URL = TARGET_URL + '/stock/3778/pick'

SAVE_FILE_NAME =  APP_CONFIG['SAVE_FILE_NAME'][os.environ['TARGET']]
DRIVER_WAIT_TIME = APP_CONFIG['DRIVER_WAIT_TIME']
MAX_SCROLL_ATTEMPTS = APP_CONFIG['MAX_SCROLL_ATTEMPTS']
SCROLL_PAUSE_TIME = APP_CONFIG['SCROLL_PAUSE_TIME']
result_dict = APP_CONFIG['RESULT_DICT']

#環境依存値読み込み
SUBMIT_FILE_PATH = APP_ENV['SUBMIT_FILE_PATH']

In [8]:
# driver設定
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument("--disable-extensions")
options.add_argument("--disable-popup-blocking")
options.add_argument(
       "user-agent=Mozilla/5.0 (X11; Linux x86_64; rv:93.0) Gecko/20100101 Firefox/93.0"
    )
options.add_experimental_option("prefs", {
    "profile.managed_default_content_settings.images": 2,  # 画像の無効化
})

driver = gs.Chrome(options=options)
driver.implicitly_wait(DRIVER_WAIT_TIME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
#カスタムロガーの設定
with open('/content/drive/MyDrive/Colab Notebooks/dev/web_scraping_crownstrategy/logging_config.json', 'r') as f:
    logger_config = json.load(f)
config.dictConfig(logger_config)
logger = getLogger('main')

try:
      driver.get(STOCK_URL)
      logger.debug(f'{STOCK_URL}：トップページへ移動しました')
except Exception as e:
      logger.error(f'{STOCK_URL}：トップページへ移動出来ませんでした: [e]')
      raise

2024-02-17 16:04:34,925 [DEBUG] main: https://minkabu.jp/stock/3778/pick：トップページへ移動しました


In [10]:
#全ページ数取得
span_element = driver.find_element(By.CSS_SELECTOR,".md_card.md_box .tar")
print(span_element.text)

# 正規表現で抽出
total_count = 1
match = re.search(r'/ 全(\d+)件', span_element.text)
if match:
    total_count = int(match.group(1))

# ページ数
num_pages = math.ceil(total_count / 20)

logger.info(f'total_count:{total_count}')
logger.info(f'num_pages:{num_pages}')


1～20件 / 全311件
2024-02-17 16:04:35,180 [INFO] main: total_count:311
2024-02-17 16:04:35,190 [INFO] main: num_pages:16


In [11]:
# 詳細ページリンク先取得
page_links = []
for page in range(1, num_pages + 1):
    TARGET_PAGE_URL = STOCK_URL + '/?page=' + str(page)
    try:
      driver.get(TARGET_PAGE_URL)
      logger.debug(f'{TARGET_PAGE_URL}：ページ{page}へ移動しました')
    except Exception as e:
        logger.error(f'{TARGET_PAGE_URL}：ページ{page}へ移動出来ませんでした: [{e}]')
        continue

    script = """
        var detail_page_links = [];
        document.querySelectorAll('.md_table.tline tr').forEach(function(el) {
        var urlElements = el.querySelectorAll('a');
        if (urlElements.length > 0) {
            var url = urlElements[0].href;
            detail_page_links.push(url);
        }
    });
    return detail_page_links;
    """

    detail_page_links = driver.execute_script(script)
    logger.info(detail_page_links)
    page_links.extend(detail_page_links)

2024-02-17 16:04:39,587 [DEBUG] main: https://minkabu.jp/stock/3778/pick/?page=1：ページ1へ移動しました
2024-02-17 16:04:39,707 [INFO] main: ['https://minkabu.jp/stock/3778/pick/10000000002955539', 'https://minkabu.jp/stock/3778/pick/10000000002955139', 'https://minkabu.jp/stock/3778/pick/10000000002955077', 'https://minkabu.jp/stock/3778/pick/10000000002954716', 'https://minkabu.jp/stock/3778/pick/10000000002954595', 'https://minkabu.jp/stock/3778/pick/10000000002954442', 'https://minkabu.jp/stock/3778/pick/10000000002951902', 'https://minkabu.jp/stock/3778/pick/10000000002951358', 'https://minkabu.jp/stock/3778/pick/10000000002951308', 'https://minkabu.jp/stock/3778/pick/10000000002950834', 'https://minkabu.jp/stock/3778/pick/10000000002950050', 'https://minkabu.jp/stock/3778/pick/10000000002949912', 'https://minkabu.jp/stock/3778/pick/10000000002949768', 'https://minkabu.jp/stock/3778/pick/10000000002948918', 'https://minkabu.jp/stock/3778/pick/10000000002948343', 'https://minkabu.jp/stock/377

In [12]:
# 詳細ページ データ取得
comments_data = []

for page_link in page_links:
    try:
      driver.get(page_link)
      logger.debug(f'{page_link}：詳細ページへ移動しました')
    except Exception as e:
        logger.error(f'{page_link}：詳細ページへ移動出来ませんでした: [e]')
        continue

    script = """
        var comments = [];
        var id = (document.querySelector('.md_article_name a')?.textContent.trim()) || '-';
        var title_comment = (document.querySelector('.com_title')?.textContent.trim()) || '-';
        var sentence_comment = (document.querySelector('.mb10.fll')?.textContent.trim()) || '-';
        var comment = title_comment + (title_comment && sentence_comment ? " " : "") + sentence_comment;
        var postDate = (document.querySelector('.md_article_name span')?.textContent.trim().replace('投稿:', '')) || '-';
        var minkabuUpDown = (document.querySelector('.ribbon.ribbon_left.pick.sell')?.textContent.trim().replace('予想', '')) || '-';

        comments.push({id: id, comment: comment, postDate: postDate, minkabuUpDown: minkabuUpDown});

        return comments;
    """

    comments_data.extend(driver.execute_script(script))

# JavaScriptから取得したデータを辞書に格納
cnt = 0
for item in comments_data:
    # 取得日時
    formatted_get_time = datetime.now().strftime('%Y/%m/%d %H:%M')

    # 投稿日時
    post_date_obj = '-' if item['postDate'] == '-' else datetime.strptime(item['postDate'], '%Y/%m/%d %H:%M')
    formatted_post_date = '-' if post_date_obj == '-' else post_date_obj.strftime('%Y/%m/%d %H:%M')

    # アラート語句１
    # GPUを使用するようにspaCyに指示
    spacy.prefer_gpu()
    alert_word_1 = '-' if item['comment'] == '-' else extract_trend_comment(item['comment'])[0]

    # 色分け
    trend_flag = '-' if alert_word_1 == '-' else classify_trend_word(alert_word_1)

    result_dict['銘柄'].append(os.environ['COMPANY'])
    result_dict['ID'].append(item['id'])
    result_dict['コメント原文'].append(item['comment'])
    result_dict['返信元ID'].append('-')
    result_dict['取得日時'].append(formatted_get_time)
    result_dict['投稿日時'].append(formatted_post_date)
    result_dict['アラート語句１'].append(alert_word_1)
    result_dict['みんかぶ上下'].append(item['minkabuUpDown'])
    result_dict['色分け'].append(trend_flag)
    result_dict['取得元'].append(os.environ['TARGET'])

    cnt += 1
    logger.info(f'取得数: {cnt}/{len(comments_data)}個取得完了')


2024-02-17 16:05:21,330 [DEBUG] main: https://minkabu.jp/stock/3778/pick/10000000002955539：詳細ページへ移動しました
2024-02-17 16:05:23,951 [DEBUG] main: https://minkabu.jp/stock/3778/pick/10000000002955139：詳細ページへ移動しました
2024-02-17 16:05:25,700 [DEBUG] main: https://minkabu.jp/stock/3778/pick/10000000002955077：詳細ページへ移動しました
2024-02-17 16:05:27,081 [DEBUG] main: https://minkabu.jp/stock/3778/pick/10000000002954716：詳細ページへ移動しました
2024-02-17 16:05:29,468 [DEBUG] main: https://minkabu.jp/stock/3778/pick/10000000002954595：詳細ページへ移動しました
2024-02-17 16:05:31,479 [DEBUG] main: https://minkabu.jp/stock/3778/pick/10000000002954442：詳細ページへ移動しました
2024-02-17 16:05:33,881 [DEBUG] main: https://minkabu.jp/stock/3778/pick/10000000002951902：詳細ページへ移動しました
2024-02-17 16:05:36,076 [DEBUG] main: https://minkabu.jp/stock/3778/pick/10000000002951358：詳細ページへ移動しました
2024-02-17 16:05:39,351 [DEBUG] main: https://minkabu.jp/stock/3778/pick/10000000002951308：詳細ページへ移動しました
2024-02-17 16:05:41,017 [DEBUG] main: https://minkabu.jp/stock/3

ValueError: time data '-' does not match format '%Y/%m/%d %H:%M'

In [ ]:
import pandas as pd
df = pd.DataFrame(result_dict)
df.tail(100)

In [ ]:
# エクセルファイルとして保存
SAVE_DIR = os.path.join(SUBMIT_FILE_PATH, SAVE_FILE_NAME)
df.to_excel(SAVE_DIR, header=True, index=False)

In [ ]:
# 参考
#コメント抽出
# ult_element = driver.find_element(By.ID,"cmtlst")
# data_comment_elements = ult_element.find_elements(By.CSS_SELECTOR, 'li[data-comment]')

# for data_comment_element in data_comment_elements:
#     # ID
#     id = data_comment_element.find_element(By.CSS_SELECTOR, ".comNum").text
#     # コメント
#     comment = data_comment_element.find_element(By.CSS_SELECTOR, ".comText").text
#     #返答先ID
#     reply_to_elements = data_comment_element.find_elements(By.CSS_SELECTOR, ".comReplyTo")
#     reply_to_text = reply_to_elements[0].text if reply_to_elements else '-'

#     #結果格納
#     result_dict['ID'].append(id)
#     result_dict['コメント'].append(comment)
#     result_dict['返信元ID'].append(reply_to_text)

# print(result_dict)

# # コメント列の内容をカンマで連結
# texts = df['コメント'].str.cat(sep=', ')
# logger.info(texts)

# エクセルからデータ読み込み
# 必要なライブラリのインストール
!pip install pandas openpyxl

# pandasをインポート
import pandas as pd

# エクセルファイルの読み込み
df = pd.read_excel(SAVE_DIR)